# Adversarial Search in Games

The following code is based on the code provided by the book [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/) in http://aima.cs.berkeley.edu/python/readme.html.

### Abstract class for modeling a game

In [1]:
class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement
    legal_moves, make_move, utility, and terminal_test. You may
    override display and successors or you can inherit their default
    methods. You will also need to set the .initial attribute to the
    initial state; this can be done in the constructor."""

    def legal_moves(self, state):
        "Return a list of the allowable moves at this point."
        abstract

    def make_move(self, move, state):
        "Return the state that results from making a move from a state."
        abstract

    def utility(self, state, player):
        "Return the value of this final state to player."
        abstract

    def terminal_test(self, state):
        "Return True if this is a final state for the game."
        return not self.legal_moves(state)

    def to_move(self, state):
        "Return the player whose move it is in this state."
        return state.to_move

    def display(self, state):
        "Print or otherwise display the state."
        print state

    def successors(self, state):
        "Return a list of legal (move, state) pairs."
        return [(move, self.make_move(move, state))
                for move in self.legal_moves(state)]

    def __repr__(self):
        return '<%s>' % self.__class__.__name__

### Minimax with alpha-beta pruning implementation

Some auxiliary functions

In [2]:
def argmin(seq, fn):
    """Return an element with lowest fn(seq[i]) score; tie goes to first one.
    >>> argmin(['one', 'to', 'three'], len)
    'to'
    """
    best = seq[0]; best_score = fn(best)
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score
    print
    return best

def argmax(seq, fn):
    """Return an element with highest fn(seq[i]) score; tie goes to first one.
    >>> argmax(['one', 'to', 'three'], len)
    'three'
    """
    return argmin(seq, lambda x: -fn(x))

Minimax search function

In [3]:
def alphabeta_search(state, game, d=float('inf'), cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)

    def max_value(state, alpha, beta, depth):
        global nodes
        if cutoff_test(state, depth):
            global nodes
            nodes += 1
            return eval_fn(state, player)
        v = -float('inf')
        for (a, s) in game.successors(state):
            nodes += 1
            v = max(v, min_value(s, alpha, beta, depth+1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        global nodes
        if cutoff_test(state, depth):            
            nodes += 1
            return eval_fn(state, player)
        v = float('inf')
        for (a, s) in game.successors(state):
            nodes += 1
            v = min(v, max_value(s, alpha, beta, depth+1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v
    
    global nodes
    
    nodes = 0

    # Body of alphabeta_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state,depth: depth>d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
    action, state = argmax(game.successors(state), lambda ((a, s)): min_value(s, -float('inf'), float('inf'), 0))
    print state,"Player",player,"moved",action,"with",nodes,"nodes expanded."
    return action


### Generic playing agents

Auxiliary functions

In [4]:
def num_or_str(x):
    """The argument is a string; convert to a number if possible, or strip it.
    >>> num_or_str('42')
    42
    >>> num_or_str(' 42x ')
    '42x'
    """
    if isnumber(x): return x
    try:
        return int(x)
    except ValueError:
        try:
            return float(x)
        except ValueError:
                return str(x).strip()

def isnumber(x):
    "Is x a number? We say it is if it has a __int__ method."
    return hasattr(x, '__int__')

A player that makes a query for each move

In [5]:
def query_player(game, state):
    "Make a move by querying standard input."
    game.display(state)
    return num_or_str(raw_input('Your move? '))

A player that chooses a move at random

In [6]:
import random

def random_player(game, state):
    "A player that chooses a legal move at random."
    return random.choice(game.legal_moves(state))

A player that uses minimimax alpha-beta search

In [7]:
def alphabeta_player(game, state):
    return alphabeta_search(state, game)

A function that receives a list of players and call each player alternatively

In [8]:
def play_game(game, *players):
    "Play an n-person, move-alternating game."
    state = game.initial
    while True:
        game.display(state)
        for player in players:
            move = player(game, state)
            state = game.make_move(move, state)
            if game.terminal_test(state):
                return game.utility(state, 0)
        print

### The last-stone game

The game is played with a heap of stones. Each player take alternatively a number $n$ of stones ($1 \le n \le 3$). The player that takes the last stone wins.

An auxiliary class to define light-weight objects

In [9]:
class Struct:
    """Create an instance with argument=value slots.
    This is for making a lightweight object whose class doesn't matter."""
    def __init__(self, **entries):
        self.__dict__.update(entries)

    def __cmp__(self, other):
        if isinstance(other, Struct):
            return cmp(self.__dict__, other.__dict__)
        else:
            return cmp(self.__dict__, other)

    def __repr__(self):
        args = ['%s=%s' % (k, repr(v)) for (k, v) in vars(self).items()]
        return 'Struct(%s)' % ', '.join(args)

The following class models the last-stone game:

In [10]:
class LastStone(Game):
    def __init__(self, stones):
        self.initial = Struct(to_move=0, heap = stones)

    def legal_moves(self, state):
        "Return a list of the allowable moves at this point."
        return range(1, min(3, state.heap) + 1)

    def make_move(self, move, state):
        "Return the state that results from making a move from a state."
        return Struct(to_move = 1 - state.to_move, heap = state.heap - move)
        
    def utility(self, state, player):
        "Return the value of this final state to player."
        if state.to_move == player:
            return -1
        else:
            return 1

    def terminal_test(self, state):
        "Return True if this is a final state for the game."
        return not self.legal_moves(state)

    def to_move(self, state):
        "Return the player whose move it is in this state."
        return state.to_move

    def display(self, state):
        "Print or otherwise display the state."
        print state

    def successors(self, state):
        "Return a list of legal (move, state) pairs."
        return [(move, self.make_move(move, state))
                for move in self.legal_moves(state)]

An interactive game against the computer, can you win?

In [11]:
def playerMat(game, state):
    player = game.to_move(state)
    print state,"Player",player,"moved",(state.heap % 4) + 1,"with 1 node expanded."
    return state.heap % 4

play_game(LastStone(12), playerMat, alphabeta_player)

Struct(to_move=0, heap=12)
Struct(to_move=0, heap=12) Player 0 moved 1 with 1 node expanded.

Struct(to_move=0, heap=11) Player 1 moved 1 with 2344 nodes expanded.

Struct(to_move=0, heap=11)
Struct(to_move=0, heap=11) Player 0 moved 4 with 1 node expanded.

Struct(to_move=0, heap=7) Player 1 moved 1 with 314 nodes expanded.

Struct(to_move=0, heap=7)
Struct(to_move=0, heap=7) Player 0 moved 4 with 1 node expanded.

Struct(to_move=0, heap=3) Player 1 moved 1 with 29 nodes expanded.

Struct(to_move=0, heap=3)
Struct(to_move=0, heap=3) Player 0 moved 4 with 1 node expanded.


1

#### 1. Design an evaluation function for the last-stone game and test it

In [12]:
def eval_fn(state, player):
    m = state.to_move
    h = state.heap
    #print state,
    
    if m == 0:
        if h % 4 == 0:
            #print -1,"|",
            return -1
        else:
            #print 1,"|",
            return 1
    elif m == 1:
        if h % 4 == 0:
            #print 1,"|",
            return 1
        else:
            #print -1,"|",
            return -1
    else:
        return None
    return None
    
def smart_player(game, state):
    #print state
    return alphabeta_search(state, game, d = 3, eval_fn = eval_fn)


result = play_game(LastStone(15), smart_player, alphabeta_player)
if result == 1:
    print "Smart player wins"
else:
    print "Smart player loses"

Struct(to_move=0, heap=15)

Struct(to_move=1, heap=12) Player 0 moved 3 with 315 nodes expanded.

Struct(to_move=0, heap=11) Player 1 moved 1 with 2344 nodes expanded.

Struct(to_move=0, heap=11)

Struct(to_move=1, heap=8) Player 0 moved 3 with 315 nodes expanded.

Struct(to_move=0, heap=7) Player 1 moved 1 with 314 nodes expanded.

Struct(to_move=0, heap=7)

Struct(to_move=1, heap=4) Player 0 moved 3 with 163 nodes expanded.

Struct(to_move=0, heap=3) Player 1 moved 1 with 29 nodes expanded.

Struct(to_move=0, heap=3)

Struct(to_move=1, heap=0) Player 0 moved 3 with 13 nodes expanded.
Smart player wins


### The 3-heaps last-stone game

In this version of the game, there are three heaps instead of 1. In each turn, a player takes $n$ stones 
($1 \le n \le k$) from one of the heaps. The player that takes the last stone wins.

#### 2. Define a class that models the 3-heaps last-stone game

In [13]:
class LastStone3Heaps(Game):
    def __init__(self, k, heap1, heap2, heap3):
        self.initial = Struct(to_move = 0, k = k, heap1 = heap1, heap2 = heap2, heap3 = heap3)

    def legal_moves(self, state):
        moves = list()
        for i in range(1, min(state.k, state.heap1) + 1):
            moves.append((1,i))
        for i in range(1, min(state.k, state.heap2) + 1):
            moves.append((2,i))
        for i in range(1, min(state.k, state.heap3) + 1):
            moves.append((3,i))            
        return moves
    
    def make_move(self, move, state):
        "Move its a pair, wich contains heap, stones."
        h, s = move
        if h == 1:
            return Struct(to_move = 1 - state.to_move, k = state.k, heap1 = state.heap1 - s, heap2 = state.heap2, heap3 = state.heap3)
        elif h == 2:
            return Struct(to_move = 1 - state.to_move, k = state.k, heap1 = state.heap1, heap2 = state.heap2 - s, heap3 = state.heap3)
        elif h == 3:
            return Struct(to_move = 1 - state.to_move, k = state.k, heap1 = state.heap1, heap2 = state.heap2, heap3 = state.heap3 - s)
        else:
            return None
    
    def utility(self, state, player):
        "Return the value of this final state to player."
        if state.to_move == player:
            return -1
        else:
            return 1

    def terminal_test(self, state):
        "Return True if this is a final state for the game."
        return not self.legal_moves(state)
    
    def to_move(self, state):
        "Return the player whose move it is in this state."
        return state.to_move
    
    def display(self, state):
        "Print or otherwise display the state."
        print state
    
    def successors(self, state):
        "Return a list of legal (move, state) pairs."
        return [(move, self.make_move(move, state)) for move in self.legal_moves(state)]

#### 3. Evaluate how many states are expanded with and without alpha-beta pruning
Redefining minimax algorithm without Alpha/Beta.

In [14]:
#Define Minimax without alpha and beta
def minimax(state, game, d=float('inf'), cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)
    
    def max_value(state, alpha, beta, depth):        
        global nodes
        if cutoff_test(state, depth):
            nodes += 1
            return eval_fn(state, player)
        v = -float('inf')
        for (a, s) in game.successors(state):
            nodes += 1
            v = max(v, min_value(s, alpha, beta, depth+1))
        return v

    def min_value(state, alpha, beta, depth):        
        global nodes
        if cutoff_test(state, depth):
            nodes += 1
            return eval_fn(state, player)
        v = float('inf')
        for (a, s) in game.successors(state):
            nodes += 1
            v = min(v, max_value(s, alpha, beta, depth+1))
        return v
    
    global nodes
    
    nodes = 0
    
    # Body of alphabeta_search starts here:
    # The default test cuts off at depth d or at a terminal state
    #cutoff_test = (cutoff_test or (lambda state,depth: depth>d or game.terminal_test(state)))
    cutoff_test = lambda state, depth: False
    eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
    
    action, state = argmax(game.successors(state), lambda ((a, s)): min_value(s, -float('inf'), float('inf'), 0))
    #print "R:",action, state, eval_fn(state, player), "Nodes:",nodes
    print state,"Player",player,"moved",action,"with",nodes,"nodes expanded."
    return action

def MiniMax3Heaps(game, state):
    return minimax(state, game)


print "Player 0: Without Alpha/Beta prunning. Player 1: Alpha/Beta prunning."


result = play_game(LastStone3Heaps(2,4,4,5), MiniMax3Heaps, alphabeta_player)


if result == 1:
    print "Smart player wins"
else:
    print "Smart player loses"

Player 0: Without Alpha/Beta prunning. Player 1: Alpha/Beta prunning.
Struct(heap3=5, heap2=4, heap1=4, k=2, to_move=0)

Struct(heap3=3, heap2=4, heap1=4, k=2, to_move=1) Player 0 moved (3, 2) with 3667937 nodes expanded.

Struct(heap3=3, heap2=4, heap1=3, k=2, to_move=0) Player 1 moved (1, 1) with 21959 nodes expanded.

Struct(heap3=3, heap2=4, heap1=3, k=2, to_move=0)

Struct(heap3=3, heap2=4, heap1=1, k=2, to_move=1) Player 0 moved (1, 2) with 84614 nodes expanded.

Struct(heap3=3, heap2=4, heap1=0, k=2, to_move=0) Player 1 moved (1, 1) with 1410 nodes expanded.

Struct(heap3=3, heap2=4, heap1=0, k=2, to_move=0)

Struct(heap3=3, heap2=3, heap1=0, k=2, to_move=1) Player 0 moved (2, 1) with 711 nodes expanded.

Struct(heap3=3, heap2=2, heap1=0, k=2, to_move=0) Player 1 moved (2, 1) with 221 nodes expanded.

Struct(heap3=3, heap2=2, heap1=0, k=2, to_move=0)

Struct(heap3=3, heap2=0, heap1=0, k=2, to_move=1) Player 0 moved (2, 2) with 97 nodes expanded.

Struct(heap3=2, heap2=0, heap1=0

#### 4. Design an evaluation function for the 3-heaps last-stone game and test it
Our evaluation fuction is based on the sum of each heap and it's module with K.

In [16]:
def fAux(m,a,b,c):
    plus = 0
    if a % m == 0:
        plus += 1
    if b % m == 0:
        plus += 1
    if c % m == 0:
        plus += 1
    return plus
        

def eval3Heaps(state, player):
        
    m = state.to_move
    h1 = state.heap1
    h2 = state.heap2
    h3 = state.heap3
    k = state.k + 1
        
    if m == 0:
        if fAux(k,h1,h2,h3) == 2:
            return 1
        else:
            return -1
    elif m == 1:
        if fAux(k,h1,h2,h3) == 2:
            return -1
        else:
            return 1
    else:
        return None
    return None
    
    
def smart3Heaps(game, state):
    return alphabeta_search(state, game, d = 2, eval_fn = eval3Heaps)

def query3Heaps(game, state):
    "Make a move by querying standard input."
    a = num_or_str(raw_input('Heap?: '))
    b = num_or_str(raw_input('Stones?: '))
    #print (a, b)
    return (a, b)


#alphabeta_player query3Heaps LastStone3Heaps
result = play_game(LastStone3Heaps(3,4,6,8), smart3Heaps, alphabeta_player)

if result == 1:
    print "Smart player wins"
else:
    print "Smart player loses"
    

Struct(heap3=8, heap2=6, heap1=4, k=3, to_move=0)

Struct(heap3=8, heap2=4, heap1=4, k=3, to_move=1) Player 0 moved (2, 2) with 2009 nodes expanded.

Struct(heap3=8, heap2=4, heap1=3, k=3, to_move=0) Player 1 moved (1, 1) with 579908 nodes expanded.

Struct(heap3=8, heap2=4, heap1=3, k=3, to_move=0)

Struct(heap3=8, heap2=4, heap1=0, k=3, to_move=1) Player 0 moved (1, 3) with 1490 nodes expanded.

Struct(heap3=8, heap2=3, heap1=0, k=3, to_move=0) Player 1 moved (2, 1) with 10243 nodes expanded.

Struct(heap3=8, heap2=3, heap1=0, k=3, to_move=0)

Struct(heap3=8, heap2=0, heap1=0, k=3, to_move=1) Player 0 moved (2, 3) with 439 nodes expanded.

Struct(heap3=7, heap2=0, heap1=0, k=3, to_move=0) Player 1 moved (3, 1) with 314 nodes expanded.

Struct(heap3=7, heap2=0, heap1=0, k=3, to_move=0)

Struct(heap3=4, heap2=0, heap1=0, k=3, to_move=1) Player 0 moved (3, 3) with 118 nodes expanded.

Struct(heap3=3, heap2=0, heap1=0, k=3, to_move=0) Player 1 moved (3, 1) with 29 nodes expanded.

Struct